[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/pantat88/segsmaker/blob/main/Segsmaker_ComfyUI.ipynb)

### Conda

In [ ]:
from IPython.display import HTML, display
import subprocess
def ass(command, message, color):
    display(HTML(f"<span style='color:{color};'>--- {message} ---</span>"))
    result = subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    display(HTML("<span style='color:#66ff00;'>done</span>"))
ass('conda install -q -y conda=23.10.0 glib gxx_linux-64 pyngrok ffmpeg av gst-libav imageio imageio-ffmpeg', 'Installing Conda', '#D48900')
ass('conda install -q -y -n base python=3.10.12', 'Installing Python 3.10.12', '#D48900')
ass('conda clean -y --all', 'Cleaning Conda environment', 'magenta')
ass('curl -Lo ~/.ipython/profile_default/startup/pantat88.py https://github.com/pantat88/segsmaker/raw/main/ui/pantat88.py', 'Gathering Magic', 'red')
!python --version
import os
os._exit(00)

### Clone ComfyUI

In [ ]:
%say Cloning ComfyUI. . . cyan
%cd -q /home/studio-lab-user
!git clone -q https://github.com/comfyanonymous/ComfyUI

%say Installing requirements. . . #D48900
%cd -q ~/ComfyUI
!pip install https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl -r requirements.txt torchvision --extra-index-url https://download.pytorch.org/whl/cu118 > /dev/null

%say Creating a symlink to temporary folder. . . magenta
!rm -rf ~/tmp ~/ComfyUI/models/checkpoints ~/ComfyUI/models/loras ~/ComfyUI/models/controlnet ~/.cache/*
!ln -vs /tmp ~/tmp
!ln -vs /tmp/loras ~/ComfyUI/models/loras
!ln -vs /tmp/checkpoints ~/ComfyUI/models/checkpoints
!ln -vs /tmp/controlnet ~/ComfyUI/models/controlnet
%say done #66ff00

### ComfyUI Custom Node

In [ ]:
%say Cloning ComfyUI Custom Node. . . cyan
%cd -q ~/ComfyUI/custom_nodes

!git clone -q https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive
!git clone -q https://github.com/ltdrdata/ComfyUI-Manager
!git clone -q https://github.com/pythongosssss/ComfyUI-Custom-Scripts
!git clone -q https://github.com/city96/SD-Latent-Upscaler
!git clone -q https://github.com/jags111/efficiency-nodes-comfyui
!git clone -q https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb
!git clone -q https://github.com/ssitu/ComfyUI_NestedNodeBuilder
!git clone -q https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet
!git clone -q https://github.com/Fannovel16/comfyui_controlnet_aux
!git clone -q https://github.com/SLAPaper/ComfyUI-Image-Selector
!git clone -q https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes
!git clone -q https://github.com/FizzleDorf/ComfyUI_FizzNodes
!git clone -q https://github.com/WASasquatch/was-node-suite-comfyui
!git clone -q https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite

!git clone -q https://github.com/Gourieff/comfyui-reactor-node
%download https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth ~/ComfyUI/models/facerestore_models
%download https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth ~/ComfyUI/models/facerestore_models

!git clone -q https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved
%download https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/models
%cd -q ~/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingAnticlockwise.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingClockwise.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt
%download https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt

!git clone -q https://github.com/cubiq/ComfyUI_IPAdapter_plus ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
%cd -q ~/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/models
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors
%download https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors ~/ComfyUI/models/clip_vision 1-5-image-encoder.safetensors

%say Installing Custom Node dependencies. . . #D48900
%cd -q ~/ComfyUI
!pip install opencv-python==4.7.0.72 transformers==4.26.1 tokenizers clip-interrogator > /dev/null
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" > /dev/null

!rm -rf ~/.cache/*
%say done #66ff00
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### *Embeddings - Upscaler - VAE*

In [ ]:
# embeddings
%cd -q ~/ComfyUI/models/embeddings
%download https://huggingface.co/pantat88/ui/resolve/main/embeddings.zip
!unzip -o embeddings.zip
!rm embeddings.zip

%download https://civitai.com/api/download/models/150491 edgQuality.pt

# Upscaler
%cd -q ~/ComfyUI/models/upscale_models

%download https://huggingface.co/pantat88/ui/resolve/main/4x-UltraSharp.pth
%download https://huggingface.co/pantat88/ui/resolve/main/4x-AnimeSharp.pth
%download https://huggingface.co/pantat88/ui/resolve/main/4x_RealisticRescaler_100000_G.pth
%download https://huggingface.co/pantat88/ui/resolve/main/8x_RealESRGAN.pth

# VAE
%cd -q ~/ComfyUI/models/vae

%download https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
#%download https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/kl-f8-anime2_fp16.safetensors
#%download https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/blessed2_fp16.safetensors
#%download https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/ClearVAE_V2.3_fp16.safetensors
#%download https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/anything_fp16.safetensors


### *ControlNET models*
###### temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p /tmp/controlnet
%cd -q ~/ComfyUI/models/controlnet

%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml
%download https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors
%download https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml

### *Checkpoint*
###### temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p ~/tmp/checkpoints
%cd -q ~/ComfyUI/models/checkpoints

%download https://civitai.com/api/download/models/147913 meichidarkMixFp16_45.safetensors


### *Lora*
###### temporary

In [ ]:
!rm -rf ~/tmp/tmp && mkdir -p /tmp/loras
%cd -q ~/ComfyUI/models/loras

%download https://civitai.com/api/download/models/87153  add_detail.safetensors
%download https://civitai.com/api/download/models/171989 detail_slider_v4.safetensors
%download https://civitai.com/api/download/models/62833  Detail_Tweaker.safetensors
%download https://civitai.com/api/download/models/86247  flat2.safetensors


### Run ComfyUI

In [ ]:
ngrok_token="PASTE_TOKEN_HERE"

from threading import Timer
from queue import Queue
from pyngrok import ngrok

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)

ngrok_output_queue = Queue()
ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
ngrok_thread.start()
ngrok_thread.join()
print(ngrok_output_queue.get())

%cd -q ~/ComfyUI
!python main.py --dont-print-server

### Output images
```zip``` output images

In [ ]:
%%zipping

input_folder  = '/home/studio-lab-user/ComfyUI/output'
output_folder = '/home/studio-lab-user'

### For checking and cleaning Storage

In [ ]:
# Check Storage & List all folders

!df -h | grep -E 'Avail|home'
print ()
!du -h --max-depth=1 /home/studio-lab-user

In [ ]:
# DELETE output Images

!rm -rf ~/ComfyUI/output/*

In [ ]:
# DELETE ComfyUI

!unlink ~/tmp
!unlink ~/ComfyUI/models/checkpoints
!unlink ~/ComfyUI/models/loras
!unlink ~/ComfyUI/models/controlnet
!rm -rf ~/ComfyUI ~/tmp ~/.cache/*

In [ ]:
# DELETE EVERYTHING , restart jupyterlab afterward

!rm -rf ~/.cache/* ~/.conda/*
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*